In [1]:
import tensorflow as tf
import numpy as np

2022-10-19 18:45:25.809254: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
print(tf.__version__)

2.5.0


In [3]:
import pickle
with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix.pkl', 'rb') as f:    
    user_dict = pickle.load(f)
items_num=max(max(user_dict.values()))
maxlen=100
len_seq=50
batch_size=512
epoch_num=20
hidden_size=64
keep_rate=0.9
layers_num=5
num_interest=1
neg_num=4
test_neg_num=100

In [4]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(hist)<maxlen):
            hist.insert(0,0)
        #print(len(hist),hist)
        hist=hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [5]:
train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)

289920 6040 6040 6040


In [6]:
users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}

In [7]:
print(np.shape(train_data['neg_item']))

(289920, 4)


In [8]:
def scaled_dot_product_attention(q, k, v, mask):
    """Attention Mechanism Function.
    Args:
        :param q: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param k: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param v: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param mask: A 3d/4d tensor with shape of (None, ..., seq_len, 1)
    :return:
    """
    mat_qk = tf.matmul(q, k, transpose_b=True)  # (None, seq_len, seq_len)
    # Scaled
    dk = tf.cast(k.shape[-1], dtype=tf.float32)
    scaled_att_logits = mat_qk / tf.sqrt(dk)

    paddings = tf.ones_like(scaled_att_logits) * (-2 ** 32 + 1)  # (None, seq_len, seq_len)
    if mask!=None:
        outputs = tf.where(tf.equal(mask, tf.zeros_like(mask)), paddings, scaled_att_logits)  # (None, seq_len, seq_len)
    else:
        outputs=scaled_att_logits
    # softmax
    outputs = tf.nn.softmax(logits=outputs)  # (None, seq_len, seq_len)
    outputs = tf.matmul(outputs, v)  # (None, seq_len, dim)

    return outputs


def split_heads(x, seq_len, num_heads, depth):
    """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    Args:
        :param x: A Tensor with shape of [batch_size, seq_len, num_heads * depth]
        :param seq_len: A scalar(int).
        :param num_heads: A scalar(int).
        :param depth: A scalar(int).
    :return: A tensor with shape of [batch_size, num_heads, seq_len, depth]
    """
    x = tf.reshape(x, (-1, seq_len, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

def normalize_adj_tensor(adj, seq_len):
    adj = adj + tf.expand_dims(tf.eye(seq_len), axis=0)
    rowsum = tf.reduce_sum(adj, axis=1)
    d_inv_sqrt = tf.pow(rowsum, -0.5)
    candidate_a = tf.zeros_like(d_inv_sqrt)
    d_inv_sqrt = tf.where(tf.math.is_inf(d_inv_sqrt), candidate_a, d_inv_sqrt)
    d_mat_inv_sqrt = tf.linalg.diag(d_inv_sqrt)
    norm_adg = tf.matmul(d_mat_inv_sqrt, adj)
    return norm_adg

In [9]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        """Multi Head Attention Mechanism.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads. If num_heads == 1, the layer is a single self-attention layer.
        :return:
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.wq = tf.keras.layers.Dense(d_model, activation=None)
        self.wk = tf.keras.layers.Dense(d_model, activation=None)
        self.wv = tf.keras.layers.Dense(d_model, activation=None)

    def call(self, q, k, v, mask):
        q = self.wq(q)  # (None, seq_len, d_model)
        k = self.wk(k)  # (None, seq_len, d_model)
        v = self.wv(v)  # (None, seq_len, d_model)
        # split d_model into num_heads * depth
        seq_len, d_model = q.shape[1], q.shape[2]
        q = split_heads(q, seq_len, self.num_heads, q.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        k = split_heads(k, seq_len, self.num_heads, k.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        v = split_heads(v, seq_len, self.num_heads, v.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        # mask
        if mask!=None:
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_heads, 1, 1])  # (None, num_heads, seq_len, 1)
        # attention
        scaled_attention = scaled_dot_product_attention(q, k, v, mask)  # (None, num_heads, seq_len, d_model // num_heads)
        # reshape
        outputs = tf.reshape(tf.transpose(scaled_attention, [0, 2, 1, 3]), [-1, seq_len, d_model])  # (None, seq_len, d_model)
        return outputs


class FFN(tf.keras.layers.Layer):
    def __init__(self, hidden_unit, d_model):
        """Feed Forward Network.
        Args:
            :param hidden_unit: A scalar.
            :param d_model: A scalar.
        :return:
        """
        super(FFN, self).__init__()
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., layer_norm_eps=1e-6):
        """Encoder Layer.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads.
            :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
            :param dropout: A scalar. Number of dropout.
            :param layer_norm_eps: A scalar. Small float added to variance to avoid dividing by zero.
        :return:
        """
        super(TransformerEncoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        x, mask= inputs
        # self-attention
        att_out = self.mha(x, x, x, mask)  # (None, seq_len, d_model)
        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)  # (None, seq_len, d_model)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2

class gcn_layer(tf.keras.layers.Layer):
    def __init__(self, hidden_size, num_layer):
        super(gcn_layer, self).__init__()
        #weights_size_list = [hidden_size,64,64,64]
        #all_weights = {}
        self.num_layer=num_layer
        self.conv_dense=tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
    
    def call(self,A, x):
    # gcn has three layers
        all_embeddings =[x]
        for k in range(self.num_layer):
            embeddings = tf.matmul(A, x)
            embeddings = self.conv_dense(embeddings)
            all_embeddings.append(embeddings)
        return all_embeddings

In [10]:
class mgnm(tf.keras.models.Model):
    def __init__(self):
        super(mgnm, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=False
        
        self.len_seq=len_seq
        self.item_embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=self.len_seq,
                                                       embeddings_initializer='random_normal',
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.pos_embedding = tf.keras.layers.Embedding(len_seq,hidden_size,input_length=1,
                                                       embeddings_initializer='random_normal',
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.dropout = tf.keras.layers.Dropout(1-keep_rate)
        
        self.encoder_layer_dict={}
        for i in range(layers_num+1):
            self.encoder_layer_dict['encoder_layer_'+str(i)]=[TransformerEncoder(hidden_size, num_heads, 128,
                                                 (1-keep_rate), layer_norm_eps) for _ in range(blocks)]
        
        self.gcn=gcn_layer(hidden_size, layers_num)
        # norm
        self.use_l2norm = use_l2norm
        
    
    def call(self, inputs):
        seq_embed=self.item_embedding(inputs['click_seq'])
        
        adj_l = tf.tile(tf.expand_dims(seq_embed, axis=2), [1, 1, len_seq, 1])
        adj_r = tf.tile(tf.expand_dims(seq_embed, axis=1), [1, len_seq, 1, 1])
        adj = tf.nn.sigmoid(tf.reduce_sum(adj_l * adj_r, axis=-1))
        self.adj_l1 = tf.norm(adj, ord=1)
        if 1:
            adj = normalize_adj_tensor(adj, len_seq) 
        all_embeddings=self.gcn(adj,seq_embed)
        pos_encoding = tf.expand_dims(self.pos_embedding(tf.range(self.len_seq)), axis=0)  # (1, seq_len, embed_dim)
        att_outputs_dict={}
        for i,element in enumerate(all_embeddings):
            att_outputs_dict['embed_'+str(i)]=element+pos_encoding
        
        
        for i in range(layers_num+1):
            for block in self.encoder_layer_dict['encoder_layer_'+str(i)]:
                att_outputs_dict['embed_'+str(i)] = block([att_outputs_dict['embed_'+str(i)],None])
        interest_dict={}
        for i in range(layers_num+1):
            interest_dict['interest_'+str(i)]=tf.slice(att_outputs_dict['embed_'+str(i)], begin=[0, self.len_seq-1, 0], size=[-1, 1, -1])
        
        
        mixed_interest=tf.concat([interest_dict['interest_'+str(i)] for i in range(layers_num+1)],axis=-2)#[b,3,h]
        #mixed_interest=interest_emb_3
        #mixed_interest=tf.squeeze(mixed_interest,axis=1)
        
        self.mixed_interest=mixed_interest
        
        if self.use_l2norm:
            pos_emb = tf.math.l2_normalize(pos_emb, axis=-1)
            neg_emb = tf.math.l2_normalize(neg_emb, axis=-1)
            user_info = tf.math.l2_normalize(user_info, axis=-1)

        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))#[b,h]
        expanded_target_item=tf.expand_dims(pos_emb,axis=1)#[b,1,h]

        alpha=tf.matmul(expanded_target_item,tf.transpose(mixed_interest,[0,2,1]))#[b,1,3]
        alpha=tf.nn.softmax(alpha)
        

        res = tf.matmul(alpha,mixed_interest)#[b,1,h]
        pos_score=tf.reduce_sum(tf.multiply(res,expanded_target_item),axis=-1)#[b,1]

        neg_emb=self.item_embedding(inputs['neg_item'])#[b,neg_num,h]
        alpha=tf.matmul(neg_emb,tf.transpose(mixed_interest,[0,2,1]))#[b,neg_num,3]
        alpha=tf.nn.softmax(alpha)
        
        res = tf.matmul(alpha,mixed_interest)#[b,neg_num,h]
        neg_score=tf.reduce_sum(tf.multiply(res,neg_emb),axis=-1)#[b,neg_num]
        
        
        
        
        
        pos_score = tf.tile(pos_score, [1, neg_score.shape[1]])
        
        loss = tf.reduce_mean(- tf.math.log(tf.nn.sigmoid(pos_score)) - tf.math.log(1 - tf.nn.sigmoid(neg_score))) / 2
        self.add_loss(loss)
        logits = tf.concat([pos_score, neg_score], axis=-1)
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'neg_item': tf.keras.layers.Input(shape=(1,), dtype=tf.int32)  # suppose neg_num=1
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [11]:
tf.keras.backend.clear_session()

model = mgnm()
#model.summary()
optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer=optimizer)

2022-10-19 18:45:34.611942: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-10-19 18:45:34.650587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:56:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2022-10-19 18:45:34.650610: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-10-19 18:45:34.653761: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-10-19 18:45:34.653819: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-10-19 18:45:34.654726: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.

In [12]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    pred_y = - model.predict(test_data, batch_size)
    return eval_rank(pred_y, metric_names, k)

In [13]:
from time import time
for epoch in range(1, epoch_num + 1):
    t1 = time()
    model.fit(
            x=train_data,
            epochs=1,
            batch_size=batch_size)
    t2 = time()
    eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))

2022-10-19 18:45:36.038811: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-19 18:45:36.041141: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2000000000 Hz
2022-10-19 18:45:46.759362: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-10-19 18:45:47.593111: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-10-19 18:45:47.593317: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-10-19 18:45:47.601207: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-10-19 18:45:48.407464: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101


567/567 [==============================] - 47s 58ms/step - loss: 0.3384
Iteration 1 Fit [47.5 s], Evaluate [2.1 s]: HR = 0.1642, MRR = 0.0619, NDCG = 0.0859
567/567 [==============================] - 34s 59ms/step - loss: 0.1469
Iteration 2 Fit [33.9 s], Evaluate [0.4 s]: HR = 0.2020, MRR = 0.0725, NDCG = 0.1029
567/567 [==============================] - 33s 57ms/step - loss: 0.1074
Iteration 3 Fit [32.7 s], Evaluate [0.4 s]: HR = 0.2205, MRR = 0.0784, NDCG = 0.1116
567/567 [==============================] - 33s 59ms/step - loss: 0.0905
Iteration 4 Fit [33.6 s], Evaluate [0.4 s]: HR = 0.2432, MRR = 0.0894, NDCG = 0.1254
567/567 [==============================] - 33s 57ms/step - loss: 0.0797
Iteration 5 Fit [32.8 s], Evaluate [0.4 s]: HR = 0.2462, MRR = 0.0908, NDCG = 0.1273
567/567 [==============================] - 33s 58ms/step - loss: 0.0719
Iteration 6 Fit [33.1 s], Evaluate [0.4 s]: HR = 0.2584, MRR = 0.0991, NDCG = 0.1366
567/567 [==============================] - 32s 57ms/step -

KeyboardInterrupt: 

In [ ]:
a=tf.constant([[2558, 2559, 2582, 2605, 2608, 2669, 2801, 2835, 2913, 3052]])
tf.print(tf.argmax(model(a)[0]))

In [ ]:
'''def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()
        
        # 在后期降低学习率
        if epoch == 5000:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")
        
        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)
            
        if epoch%100 ==0:
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")'''

In [ ]:
'''train_model(model,ds_train,ds_test,800)'''